This notebook fetches and formats data relevant to the comprehensive list of farms and gardens in NYC. Sources include:

* GreenThumb gardens (NYC OpenData)
* NYCHA gardens (provided by Parks)
* School gardens (provided by GrowNYC)
* Potential garden locations (Local Law 46, 2018, via NYC OpenData)
* Data from ~100 large farms and gardens (survey responses plus data from public websites and reports)
* Community gardens from public sources (GrowNYC, Brooklyn Queens Land Trust, Bronx Land Trust, Manhattan Land Trust)

Link to Google Drive where all flat files are stored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Path below will be unique to your Drive

In [ ]:
cd '/content/drive/My Drive/Summer/MAP NYC x CUSP/data/urban_agriculture'

/content/drive/My Drive/Summer/MAP NYC x CUSP/data/urban_agriculture


Install and import required packages (Colab has many pre-installed)

In [ ]:
!pip install geopandas rtree

     |████████████████████████████████| 1.0 MB 7.8 MB/s 
     |████████████████████████████████| 994 kB 44.9 MB/s 
     |████████████████████████████████| 6.3 MB 40.2 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rtree

import requests
from urllib.request import urlretrieve
from zipfile import ZipFile
from os import remove as remove_file
from shutil import rmtree as remove_dir

from geopy.geocoders import GoogleV3

Define API handler for NYC Open Data

In [ ]:
def get_socrata(resource, query='', dates=[], limit=1000, sample=False, domain='data.cityofnewyork.us', progress=False):
    headers = {'X-App-Token': 'b0nu5h6f5RSJRNPajPuuCf7wo'}
    result = []
    offset = 0
    while offset >= 0:
        url = f'https://{domain}/resource/{resource}.json?$limit={limit}&$offset={offset}&$order=:id'
        if query != '': #can't use requests params bc of possible $ char
            url += f'&{query}'
        if (progress):
            print(url)
        json = requests.get(url, headers=headers).json()
        if 'error' in json:
            print(json['message'])
            return
        else:
            df = pd.DataFrame(json)
            if len(df):
                if len(dates):
                    df[dates] = df[dates].apply(pd.to_datetime, errors='coerce')
                result.append(df)
                offset += limit
            else:
                offset = -1
        if sample:
          break

    result = pd.concat(result).reset_index() if len(result) else pd.DataFrame()
    print(f'Retreived {len(result)} records')
    return result

Get Green Thumb data from [Open Data](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=GreenThumb+Gardens&q=greenthumb)

In [ ]:
gt_info = get_socrata('p78i-pat6')
gt_info.head()

Retreived 589 records


,index,assemblydist,borough,communityboard,congressionaldist,coundist,gardenname,juris,multipolygon,parksid,policeprecinct,statesenatedist,status,zipcode,openhrssa,openhrssu,openhrsf,openhrsth,openhrstu,openhrsw,openhrsm
0,0,78,X,207,13,11,Botanical Square Community Garden,MTA,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",XGT084,52,34,Not GreenThumb,10458,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,X,210,14,13,NYCHA - Throggs Neck - Love Community Garden,NYCHA,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",XGT083,45,34,Active,10465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,84,X,201,15,8,NYCHA - Moore - El Jardín de Los Amigos de Moore,NYCHA,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",XGT082,40,29,Active,10455,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,79,X,203,15,16,NYCHA - McKinley - Joyful Garden,NYCHA,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",XGT081,42,32,Active,10456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,80,X,211,14,13,NYCHA - Pelham - Pelham Organics,NYCHA,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",XGT080,49,34,Active,10469,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gt_bbl = get_socrata('fsjc-9fyh')
gt_bbl.head()

Retreived 1028 records


,index,parksid,block,lotnum,lotsize,areacovered
0,0,B024-GT001,2912,1,NaN,NaN
1,1,B047A-GT001,3889,1,17440.0,NaN
2,2,B058-GT001,2696,1,6095.0,Partial Lot
3,3,B058-GT002,2696,1,NaN,NaN
4,4,B088-GT001,1790,1,NaN,NaN


In [ ]:
gt_visit = get_socrata('xqbk-beh5')
gt_visit.head()

Retreived 354 records


,index,parksid,inspectionid,nexttoanothergarden,totalfencelength,totalsidewalkarea,totalsidewalklength,onsiteservice,hydrantw_in15ft,hydrantongardenside,rainharvesting,raingallons,solarpanels,compostsystem,composttumblers,nonfoodplants,food,openlawnorcommunalarea,pavedarea,treesingarden,fruittrees,streettrees,emptytreepits,murals,blankshed,parkssign,chickens,pond,turtles,aquaponics,farmersmarket,csapickup,composting,greenhouse,structureforseasonextension,fishinpond
0,0,B544-GT001,1,False,213.25,276.0,23.0,False,False,True,True,500,False,True,False,True,True,True,True,True,False,False,False,False,True,False,True,False,False,False,False,False,True,True,False,NaN
1,1,M397-GT001,3,False,25.0,300.0,25.0,False,True,False,True,200,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,NaN
2,2,M331-GT002,5,True,15.5,160.0,10.0,False,False,False,True,60,False,False,False,True,False,True,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,NaN
3,3,M297A-GT001,8,False,204.0,910.0,70.0,False,False,True,False,NaN,False,False,True,True,True,True,False,True,True,True,False,False,False,True,False,False,False,False,False,False,True,False,False,NaN
4,4,X277-GT001,9,True,265.0,3750.0,150.0,False,False,True,False,NaN,False,False,True,True,True,True,False,True,True,True,True,True,True,True,False,False,False,False,False,False,True,True,False,NaN


Get PLUTO data from [DCP](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page) (much faster than using Socrata API)

In [ ]:
ver = '21v2'
url = f'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_{ver}_csv.zip'

tmp = 'tmp/'
zip = 'tmp.zip'
urlretrieve(url, zip)
with ZipFile(zip, 'r') as zip_ref:
    zip_ref.extractall(tmp)

usecols = ['borocode', 'block', 'lot', 'latitude', 'longitude', 'address', 'zipcode', 'yearbuilt', 'lotarea', 'ownername', 'ownertype', 'landuse']
pluto = pd.read_csv(f'{tmp}pluto_{ver}.csv', usecols=usecols)

remove_dir(tmp)
remove_file(zip)

pluto.head()

,block,lot,zipcode,address,landuse,ownertype,ownername,lotarea,yearbuilt,borocode,latitude,longitude
0,4660,34,10467.0,CARLISLE PLACE,9.0,X,NEW YORK GARDEN TRUST INC,5000.0,0.0,2,40.877873,-73.863604
1,445,54,10003.0,65 EAST 3 STREET,2.0,NaN,65 EAST 3RD STREET LLC,1923.0,1999.0,1,40.725333,-73.988711
2,467,55,10003.0,213 EAST 11 STREET,2.0,NaN,TRES REALTY LLC,2550.0,1920.0,1,40.731190,-73.987721
3,2621,7501,11222.0,636 LEONARD STREET,2.0,NaN,UNAVAILABLE OWNER,2500.0,2007.0,3,40.726484,-73.951001
4,1970,58,10027.0,106 CONVENT AVENUE,2.0,NaN,"106-108 CONVENT BCR, LLC",7200.0,1901.0,1,40.817053,-73.952741


In [ ]:
pluto['address'] = (
    pluto['address'].str.title() +
    ', ' +
    pluto['borocode'].map({1: 'New York', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'}) + 
    ', NY ' + 
    pluto['zipcode'].fillna('').astype(str).str.replace('\\.0$', '') + 
    ', USA'
)

Combine GT and PLUTO datasets for BBL/addresses/coordinates of gardens

In [ ]:
gt = gt_info.merge(gt_bbl, on='parksid').merge(gt_visit, how='left', on='parksid')
gt['status'].value_counts()

Active                      953
Not GreenThumb               42
Closed (Other)               10
Inactive (No Group)           9
Closed (Construction)         7
Active (Unlicensed)           6
Inactive (Group Forming)      3
Name: status, dtype: int64

In [ ]:
gt['borocode'] = gt['borough'].map({'M': 1, 'X': 2, 'B': 3, 'Q': 4, 'R': 5})
gt.rename(columns={'lotnum': 'lot'}, inplace=True)

bbl_cols = ['borocode', 'block', 'lot']
gt[bbl_cols] = gt[bbl_cols].apply(pd.to_numeric)

gt = (
    gt.merge(pluto, on=bbl_cols)
    .rename(columns={'gardenname': 'name', 'latitude': 'lat', 'longitude': 'lon'})
    .sort_values(['status', *bbl_cols, 'name'])
)

gt['type'] = 'Community farm or garden'
gt['grow'] = 'Outdoor'
gt['source'] = 'GreenThumb'
len(gt)

1020

In [ ]:
gt_keep = gt[gt['status'].isin(['Active', 'Not GreenThumb']) & (gt['juris'] != 'NYCHA')] # assume NYCHA set below is complete and more accurate

In [ ]:
gt_keep

,index_x,assemblydist,borough,communityboard,congressionaldist,coundist,name,juris,multipolygon,parksid,policeprecinct,statesenatedist,status,zipcode_x,openhrssa,openhrssu,openhrsf,openhrsth,openhrstu,openhrsw,openhrsm,index_y,block,lot,lotsize,areacovered,index,inspectionid,nexttoanothergarden,totalfencelength,totalsidewalkarea,totalsidewalklength,onsiteservice,hydrantw_in15ft,hydrantongardenside,rainharvesting,raingallons,solarpanels,compostsystem,composttumblers,nonfoodplants,food,openlawnorcommunalarea,pavedarea,treesingarden,fruittrees,streettrees,emptytreepits,murals,blankshed,parkssign,chickens,pond,turtles,aquaponics,farmersmarket,csapickup,composting,greenhouse,structureforseasonextension,fishinpond,borocode,zipcode_y,address,landuse,ownertype,ownername,lotarea,yearbuilt,lat,lon,type,grow,source
521,324,65,M,101,10,1,Fishbridge Park Garden,DPR,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",M291-GT001,1,26,Active,10038,7:00a - 7:00p,7:00a - 7:00p,NaN,NaN,NaN,NaN,NaN,499,106,22.0,4034.0,Full Lot,115.0,201,False,211.0,824.0,191.0,True,True,True,False,NaN,False,False,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,NaN,1,10038.0,"Pearl Street, New York, NY 10038, USA",9.0,C,NYC DEPARTMENT OF PARKS AND RECREATION,4034.0,0.0,40.709068,-74.001378,Community farm or garden,Outdoor,GreenThumb
328,209,65,M,103,7,1,Lower East Side People Care,MLT,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",MGT024,7,26,Active,10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,692,271,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10002.0,"25 Rutgers Street, New York, NY 10002, USA",9.0,X,THE MANHATTAN LAND TRUST,2135.0,0.0,40.713006,-73.989824,Community farm or garden,Outdoor,GreenThumb
443,286,65,M,103,12,1,Clinton Community Garden (LES),DPR,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",M335-GT001,7,26,Active,10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,577,344,148.0,NaN,NaN,53.0,111,False,151.0,1452.0,121.0,False,True,True,False,NaN,False,False,False,True,True,True,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,NaN,1,10002.0,"Stanton Street, New York, NY 10002, USA",9.0,C,NYC DEPARTMENT OF PARKS AND RECREATION,2533.0,0.0,40.720010,-73.984459,Community farm or garden,Outdoor,GreenThumb
356,229,65,M,103,12,1,Siempre Verde Garden,DPR,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",M408-GT001,7,26,Active,10002,10:00a - 6:00p,10:00a - 6:00p,NaN,NaN,NaN,NaN,NaN,663,344,153.0,NaN,NaN,285.0,484,False,180.0,550.0,50.0,False,True,True,True,50,False,True,False,True,True,True,False,True,True,False,False,True,False,True,False,False,False,False,False,False,True,False,False,NaN,1,10002.0,"Stanton Street, New York, NY 10002, USA",9.0,C,NYC DEPARTMENT OF PARKS AND RECREATION,1197.0,0.0,40.719930,-73.984019,Community farm or garden,Outdoor,GreenThumb
357,229,65,M,103,12,1,Siempre Verde Garden,DPR,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",M408-GT001,7,26,Active,10002,10:00a - 6:00p,10:00a - 6:00p,NaN,NaN,NaN,NaN,NaN,664,344,158.0,NaN,NaN,285.0,484,False,180.0,550.0,50.0,False,True,True,True,50,False,True,False,True,True,True,False,True,True,False,False,True,False,True,False,False,False,False,False,False,True,False,False,NaN,1,10002.0,"Attorney Street, New York, NY 10002, USA",11.0,C,NYC DEPARTMENT OF PARKS AND RECREATION,1405.0,0.0,40.719744,-73.984008,Community farm or garden,Outdoor,GreenThumb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,136,38,Q,409,7,32,M.S. 210 - YMCA Beacon Center Garden,DOE,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",QGT019,102,15,Not GreenThumb,1

Load potentially suitable Green Thumb sites from [LL 48 of 2011](https://data.cityofnewyork.us/City-Government/Suitability-of-City-Owned-and-Leased-Property-for-/4e2n-s75z)

In [ ]:
ll48 = get_socrata('4e2n-s75z')
ll48.head()

Retreived 69533 records


,index,date_created,boro,block,lot,address,street_number,street_name,parcel_name,agency,current_uses,total_area,structure_completed,number_structures,total_gross_area_structures,ratio_building_to_floor_area,allowable_building_to_floor_area,land_use_category,community_district,census_tract,census_block,school_dist,council_district,zip_code,fire_comp,health_area,health_ctr,police_prct,major_use,number_of_easements,commercial_floor_area,residential_floor_area,office_floor_area,retail_floor_area,garage_floor_area,storage_floor_area,factory_floor_area,other_floor_area,num_floors,residential_units,residential_and_non_residential_units,lot_front,lot_depth,bldg_front,bldg_depth,proximity_code,irr_lot_code,lot_type_code,bsmt_code,assess_land,exempt_tot,year_alter_1,year_alter_2,condominium_number,coordinates,zone_dist_1,sp_dist_1,contact,edc___occupied,pluto_version,latitude,longitude,bin,bbl,nta,landmark,zone_dist_2,potential_urban_ag,his_dist,overlay_1,open_petroleum_spill,e_designation_number,govt_clean_up_program,sp_dist_2,overlay_2,industrial_business_zone,exempt_land,map_atlas
0,0,2019-10-10T00:00:00.000,1,2,1,4 SOUTH STREET,4,SOUTH STREET,SI FERRY TERMINAL,DOT;DSBS,FERRY TERMINAL;NO USE;WATERFRONT PROPERTY,209215,1908,2,280000,1.34,6.5,7,101,9,1025,2,1,10004,L015,7700,15,1,Y7,0,280000,0,0,0,0,0,0,280000,5,0,1,314.28,564.84,268,210,1,Y,2,0,18818100,35179200,2006,2006,0,980670/194440,M1-4,LM,Saundra Malanowicz (212)386-0612 smalanowicz@d...,100,19v1,40.700369,-74.012911,1085792,1000020001,Battery Park City-Lower Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-10-10T00:00:00.000,1,2,2,10 SOUTH STREET,10,SOUTH STREET,BATTERY MARITIME BLDG,DSBS,IN USE-TENANTED;LONG-TERM AGREEMENT;WATERFRONT...,191502,1900,1,158197,0.83,10,7,101,9,1025,2,1,10004,L015,7700,15,1,Y7,0,158197,0,0,0,0,0,0,158197,5,0,1,311.61,555.84,205,270,0,Y,2,0,18787500,26151750,2001,0,0,981037/194506,C4-6,LM,Saundra Malanowicz (212)386-0612 smalanowicz@d...,100,19v1,40.700551,-74.011588,1000003,1000020002,Battery Park City-Lower Manhattan,INDIVIDUAL LANDMARK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019-10-10T00:00:00.000,1,2,3,MARGINAL STREET,NaN,MARGINAL STREET,MTA SUBSTATION,DSBS,NO USE-NON RES STRC;TRANSIT SUBSTATION,38800,1900,1,7500,0.19,10,7,101,9,1,NaN,1,NaN,NaN,NaN,NaN,NaN,Y7,0,7500,0,0,0,0,0,0,7500,1,0,0,91.9,564.56,50,75,0,N,2,5,735750,898200,0,0,0,981224/194590,C4-6,LM,Saundra Malanowicz (212)386-0612 smalanowicz@d...,NaN,19v1,40.700781,-74.010913,1089745,1000020003,Battery Park City-Lower Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-10-10T00:00:00.000,1,2,23,1 PIER 6,1,PIER 6,PIER 6,DSBS,IN USE-TENANTED;FINAL COMMITMNT-DISP;LONG-TERM...,510025,1987,1,24346,0.05,10,7,101,9,1025,2,1,10004,L015,7700,15,1,T2,0,24346,0,0,0,0,0,0,24346,2,0,0,793.67,551,85,551,0,Y,5,0,15255450,18723600,0,0,0,981687/194732,C4-6,LM,Saundra Malanowicz (212)386-0612 smalanowicz@d...,100,19v1,40.701171,-74.009243,1075697,1000020023,Battery Park City-Lower Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-10-10T00:00:00.000,1,3,1,10 BATTERY PARK,10,BATTERY PARK,BATTERY PARK,PARKS,PARK,945425,1951,6,945425,1,0,9,101,13,2005,2,1,10004,E010,7700,15,1,Q1,0,945425,0,0,0,0,0,0,945425,1,0,4,323,1260,89,57,1,Y,1,0,127100700,141688800,0,0,0,979916/195328,PARK,NaN,Saundra Malanowicz (212)386-0612 smalanowicz@d...,NaN,19v1,40.702806,-74.015631,1077335,1000030001,Battery Park City-Lower Manhattan,INDIVIDUAL LANDMARK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ll48['potential_urban_ag'].value_counts()

Not Suitable 1 - Site is not suitable for any one or more of these reasons: land-based reasons; reserved for agency purposes; no safe access to the site; site is part of a built street/highway    7373
Not Suitable 2 - Site is smaller than 2500 square feet                                                                                                                                              4226
Existing GreenThumb Garden                                                                                                                                                                          1966
Pending                                                                                                                                                                                              444
Potentially Suitable 3 - Site has specific instructions or conditions that limit its availability and/or use                                                                                        

In [ ]:
ll48_pot = (
    ll48[ll48['potential_urban_ag'].str.contains('Potentially Suitable') == True]
    .rename(columns={'boro': 'borocode', 'parcel_name': 'name', 'latitude': 'lat', 'longitude': 'lon', 'total_area': 'lotarea'})
    .dropna(subset=['lat', 'lon'])
    .sort_values(bbl_cols)
)
ll48_pot['name'].fillna(ll48_pot['address'], inplace=True)
ll48_pot['type'] = 'Potential community farm or garden'
ll48_pot['grow'] = np.where(pd.to_numeric(ll48_pot['land_use_category'], errors='coerce') > 9, 'Outdoor', np.nan)
ll48_pot['source'] = 'LL48'

In [ ]:
ll48_pot['address'] = (
    ll48_pot['address'].str.title() +
    ', ' +
    ll48_pot['borocode'].map({'1': 'New York', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}) + 
    ', NY ' + 
    ll48_pot['zip_code'].fillna('').astype(str).str.replace('\\.0$', '') + 
    ', USA'
)

Load list of NYCHA gardens provided by Carlos Martinez at GreenThumb (NYC Parks)

In [ ]:
nycha = gpd.read_file('nycha_gardens')
nycha.head()

,Developmen,Length,Street_Acc,Size_sq_ft,Width,Fenced,Green_hous,Water_acce,GardenType,SignText,Raised_bed,Pathway,Distance_T,Composting,RaisedBedC,Comments,Sign,Shed,RainBarrel,GlobalID,RatBurrows,Garden_Nam,PlayAreaAd,geometry
0,Smith,87,No,0,54,Yes,No,Onsite,Combo,None,Yes,No,32,Yes,47,family plots are labeled with a sign,No,No,No,c6342b6f-e0ce-4269-968c-41cf1b620028,No,None,No,POINT (984564.792 197796.172)
1,Smith,105,Yes,0,47,Yes,No,Onsite,Food,None,Yes,Yes,24,No,22,"half active, fallen trees, rat borrows, lots o...",No,No,No,9c74cf12-f75d-44d3-b0ee-41cc81c38ca9,Yes,None,Yes,POINT (984743.240 198388.467)
2,Smith,16,No,0,41,Yes,No,Onsite,Flowers,None,Yes,No,68,Yes,27,Received citizens community grant,No,No,No,c73d9e48-a845-40a9-a50b-57a52817ca66,No,None,No,POINT (984541.796 198282.016)
3,Rutgers,54,No,0,52,Yes,No,Offsite,Combo,Rutgers House Tenant Garden,Yes,Yes,21,No,16,"individual signs in photos, gardeners name is ...",Yes,No,No,2603a661-47ab-4779-b90f-24e5a492cbba,Yes,Rutgers House Tenants' Garden,Yes,POINT (986857.999 198696.545)
4,Vladeck,0,No,0,0,Yes,No,None,Combo,None,No,Yes,0,No,0,None,No,No,No,58a29c13-92c7-40ca-9bc3-74a0e5a63222,No,None,Yes,POINT (988929.475 198792.785)


In [ ]:
nycha.to_crs('EPSG:4326', inplace=True)
nycha['lon'] = nycha['geometry'].x
nycha['lat'] = nycha['geometry'].y

In [ ]:
nycha['lotarea'] = nycha['Length'] * nycha['Width']
nycha.sort_values(['Developmen', 'lotarea'], ascending=False, inplace=True)
nycha['dev_index'] = nycha['Developmen'].duplicated().groupby(nycha['Developmen']).cumsum() + 1
nycha['dev_name'] = nycha['Developmen'] + ' Garden ' + nycha['dev_index'].astype(str)
nycha['name'] = nycha['Garden_Nam'].fillna(nycha['dev_name'])
nycha['type'] = 'NYCHA farm or garden'
nycha['grow'] = 'Outdoor'
nycha['source'] = 'NYCHA'

Load school gardens provided by GROW NYC

In [ ]:
school = pd.read_excel('school_gardens.xlsx')
school.head()

,Building Code,School Name,Type,Number,Address,Borough,State,Zip
0,NaN,P233Q@42,P (District 75),233,488 Beach 66th St,Arverne,NY,11692.0
1,Q811,PS811Q@ Queens High School of Teaching,PS,811,74-20 Commonwealth Blvd.,Bellerose,NY,11426.0
2,NaN,New York City Children's Center - Queens,P (District 75),023,74-03 Commonwealth Boulevard,Bellerose,NY,11426.0
3,X034,NaN,PS,076,770 GROTE STREET,BRONX,NY,10460.0
4,X154,Jonathan D. Hyatt,PS,154,333 EAST 135 STREET,BRONX,NY,10454.0


In [ ]:
school = school[~school['Address'].isna()].copy()
school['Name'] = school['School Name'].fillna(school['Type'].fillna(school['Borough']) + ' ' + school['Number'])
school['Farm Address'] = school['Address'] + ', ' + school['Borough'] + ' ' +  school['State'] + ' ' + school['Zip'].astype(str).str.replace('.0$', '')
school['Type'] = 'School farm or garden (including universities)'
school['Indoor/Outdoor'] = 'Outdoor'
school['Website'] = np.nan

Load urban farm data from Stern survey list

In [ ]:
farm = pd.read_csv('urban_farms.csv')
farm.head()

,Name,Farm Address,Contact Info,Email,Website,Type,Main Crops,Indoor/Outdoor,Exclude
0,Agripublica,NaN,info@agripublica.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Battery Farm,"State St &, Battery Pl, New York, NY 10004",gabriela.marin@thebattery.org\n(212) 344-3491,gabriela.marin@thebattery.org,https://www.thebattery.org/destinations/urban-...,Community Garden,NaN,Outdoor,NaN
2,BAY VIEW HOUSES FARM (Green City Force),"Bay View Houses, Canarsie, Brooklyn\nThe close...",managed by green city force,"Tonya Gayle <tonya@greencityforce.org>,\nlisbe...",https://greencityforce.org/service-corps/farms...,Institutional (NYCHA),NaN,Outdoor,NaN
3,Bowery Farming,"151 W 26th St 12th Floor, New York, NY 10001",pgauthier@boweryfarming.com\n609-480-2690,pgauthier@boweryfarming.com,https://boweryfarming.com/vision/,Commercial,NaN,Indoor,NaN
4,ADD: Bowery Farming,"PRODUCTION FARM(S), Kearny, NJ",NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load community garden data from curated list

In [ ]:
garden = pd.read_csv('community_gardens.csv')
garden.head()

,Name,Farm Address,Source,Founded,Rennovated,Garden Type,Area (sqft),Main Crops,Production (lbs),Raised beds,Greenhouse,Composting,Rain harvesting (gal),Bees,Chickens,Contact name,Contact email,Contact phone,Website
0,1100 Block Bergen Street Community Garden,"1107 Bergen Street, Brooklyn 11216",https://bqlt.org/garden/1100-block-bergen-st-c...,1980.0,NaN,Community Garden,NaN,NaN,NaN,NaN,NaN,Wednesday: 3-5pm,Yes,NaN,NaN,Hazel Hurley,NaN,917-612-2065,https://www.facebook.com/1100-Bergen-Street-Co...
1,227th Street Cambria Heights Community Garden,"116-98 227th St, Queens, NY 11411",https://bqlt.org/garden/227th-street-cambria-h...,NaN,NaN,Community Garden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Back To Eden Community Garden,"144-27 Lakewood Avenue, Queens, NY",https://bqlt.org/garden/back-to-eden-community...,1984.0,NaN,Community Garden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Central Bainbridge Street Community Garden,"277 Bainbridge Street, Brooklyn 11233",https://bqlt.org/garden/central-bainbridge-st-...,1978.0,NaN,Community Garden,NaN,Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Classon Ful-Gate Block Association Garden,"472 Classon Avenue, Brooklyn 11238",https://bqlt.org/garden/classon-ful-gate-block...,1968.0,NaN,Community Garden,NaN,"Fruit trees, vegetables",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lydia Jessup,classongardengroup@googlegroups.com,206-370-1865,NaN


In [ ]:
garden['Type'] = 'Community farm or garden'
garden['Indoor/Outdoor'] = 'Outdoor'

farm_keep = farm[(farm['Type'].str.contains('NYCHA') == False) & (farm['Exclude'] != 'TRUE')].copy()
farm_keep['Type'] = np.where(farm_keep['Type'].str.contains('commercial', case=False), 'Commercial farm or garden', 'Non-commercial farm or garden')

cols = {'Name': 'name', 'Type': 'type', 'Farm Address': 'address', 'Indoor/Outdoor': 'grow', 'Website': 'website'}
farm_garden = pd.concat([farm_keep[cols.keys()], garden[cols.keys()], school[cols.keys()]], ignore_index=True).rename(columns=cols)

In [ ]:
len(farm_garden)

1066

Geocode addresses

In [ ]:
geolocator = GoogleV3(api_key='[your API key here]')

In [ ]:
nycha['geo'] = nycha[['lat', 'lon']].apply(tuple, axis=1).apply(geolocator.reverse)
nycha['address'] = nycha['geo'].apply(lambda geo: geo[0].address)

In [ ]:
farm_garden['geo'] = farm_garden['address'].apply(geolocator.geocode)
farm_garden['point'] = farm_garden['geo'].apply(lambda geo: tuple(geo.point) if geo else None)
farm_garden[['lat', 'lon', 'alt']] = pd.DataFrame(farm_garden['point'].to_list(), index=farm_garden.index)

In [ ]:
sum(farm_garden['geo'].isnull())

0

In [ ]:
farm_garden['address'] = farm_garden['geo'].apply(lambda geo: geo.address) #use formatted (interpreted) address

In [ ]:
farm_garden.drop(columns=['geo', 'point', 'alt']).to_csv('farm_garden.csv', index=False)

Get lot areas by performing spatial join with [MapPLUTO](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page) in QGIS (too large for Colab) to get BBL for each location. Joined farm/garden-PLUTO data is _farm_garden_pluto.csv_

In [ ]:
fg_pluto = pd.read_csv('farm_garden_pluto.csv')
fg_pluto.columns = fg_pluto.columns.str.lower()
fg_pluto.head()

,name,type,source,address,grow,lat,lon,borough,block,lot,cd,ct2010,cb2010,schooldist,council,zipcode,firecomp,policeprct,healthcent,healtharea,sanitboro,sanitdistr,sanitsub,address_2,zonedist1,zonedist2,zonedist3,zonedist4,overlay1,overlay2,spdist1,spdist2,spdist3,ltdheight,splitzone,bldgclass,landuse,easements,ownertype,ownername,...,ext,proxcode,irrlotcode,lottype,bsmtcode,assessland,assesstot,exempttot,yearbuilt,yearalter1,yearalter2,histdist,landmark,builtfar,residfar,commfar,facilfar,borocode,bbl,condono,tract2010,xcoord,ycoord,zonemap,zmcode,sanborn,taxmap,edesignum,appbbl,appdate,plutomapid,firm07_fla,pfirm15_fl,version,dcpedited,latitude,longitude,notes,shape_leng,shape_area
0,Battery Farm,Non-commercial Farm,Stern,"State St & Battery Pl, New York, NY 10004, USA",Outdoor,40.704513,-74.014278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bowery Farming,Commercial Farm,Stern,"151 W 26th St 12th floor, New York, NY 10001, USA",Indoor,40.745785,-73.993244,MN,802.0,8.0,105.0,95.0,1003.0,2.0,3.0,10001.0,E001,13.0,15.0,5200.0,1.0,5.0,1H,151 WEST 26 STREET,M1-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,O6,5.0,0.0,NaN,ABNER PROPERTIES CO,...,N,3.0,Y,5.0,2.0,2767500.0,12857400.0,0.0,1911.0,1989.0,0.0,NaN,NaN,11.78,0.0,10.0,10.0,1.0,1.008020e+09,0.0,95.0,986113.0,211004.0,8d,NaN,105S012,10305.0,NaN,0.0,NaN,1.0,NaN,NaN,21v2,NaN,40.745834,-73.993277,NaN,0.0,1.561102e+04
2,BrightFarms,Commercial Farm,Stern,"211 E 48th St, New York, NY 10017, USA",Indoor,40.754407,-73.970885,MN,1322.0,107.0,106.0,90.0,5000.0,2.0,4.0,10017.0,L002,17.0,13.0,5000.0,1.0,6.0,3A,211 EAST 48 STREET,R8B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,O2,5.0,0.0,NaN,HENDALE NY LLC,...,E,0.0,N,5.0,0.0,149850.0,584100.0,0.0,1865.0,0.0,0.0,NaN,INDIVIDUAL LANDMARK,2.69,4.0,0.0,4.0,1.0,1.013220e+09,0.0,90.0,992327.0,214150.0,8d,NaN,104 051,10506.0,NaN,0.0,NaN,1.0,NaN,NaN,21v2,t,40.754465,-73.970847,NaN,0.0,1.729214e+03
3,Brooklyn Grange Rooftop Farms,Commercial Farm,Stern,"37-18 Northern Blvd, Long Island City, NY 1110...",Rooftop,40.751924,-73.925419,QN,214.0,40.0,401.0,171.0,1003.0,30.0,26.0,11101.0,L128,114.0,41.0,720.0,4.0,1.0,6C,37-02 NORTHERN BOULEVARD,M1-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,O5,5.0,0.0,NaN,RXR SMP OWNER LLC,...,N,0.0,Y,3.0,0.0,1889100.0,15138900.0,0.0,1919.0,2001.0,2009.0,NaN,NaN,8.33,0.0,5.0,6.5,4.0,4.002140e+09,0.0,171.0,1004946.0,213237.0,9b,NaN,401 065,40202.0,NaN,0.0,NaN,1.0,NaN,NaN,21v2,NaN,40.751939,-73.925303,NaN,0.0,5.188052e+04
4,Brooklyn Grange Rooftop Farms,Commercial Farm,Stern,"63 Flushing Ave Bldg 3, Brooklyn, NY 11205, USA",Rooftop,40.698459,-73.977756,BK,2023.0,1.0,302.0,543.0,1007.0,13.0,33.0,11251.0,E207,88.0,36.0,9300.0,3.0,2.0,NaN,652 KENT AVENUE,M3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,T2,7.0,0.0,C,NYC DEPARTMENT OF SMALL BUSINESS SERVICES,...,N,0.0,Y,5.0,0.0,32522850.0,204659100.0,204659100.0,1958.0,2004.0,0.0,NaN,INDIVIDUAL LANDMARK,1.10,0.0,2.0,0.0,3.0,3.020230e+09,0.0,543.0,991801.0,195129.0,12d,NaN,303 003,30705.0,NaN,0.0,NaN,1.0,1.0,1.0,21v2,t,40.702258,-73.972767,NaN,0.0,9.997440e+06


Combine all the datasets into a single dataframe and output as .csv for analysis in _urban_ag_social_infrastructure.ipynb_

In [ ]:
base_cols = ['name', 'address', 'type', 'grow', 'source', 'lat', 'lon', 'lotarea']
bbl_cols = ['borocode', 'block', 'lot']
full_cols = [*base_cols, *bbl_cols]

urban_ag = pd.concat([gt_keep[full_cols], fg_pluto[full_cols], ll48_pot[full_cols]]).drop_duplicates(subset=bbl_cols)
urban_ag = urban_ag.append(nycha[base_cols]).reset_index(drop=True)
urban_ag

,name,address,type,grow,source,lat,lon,lotarea,borocode,block,lot
0,Fishbridge Park Garden,"Pearl Street, New York, NY 10038, USA",Community Garden,Outdoor,GreenThumb,40.7091,-74.0014,4034,1,106,22
1,Lower East Side People Care,"25 Rutgers Street, New York, NY 10002, USA",Community Garden,Outdoor,GreenThumb,40.713,-73.9898,2135,1,271,63
2,Clinton Community Garden (LES),"Stanton Street, New York, NY 10002, USA",Community Garden,Outdoor,GreenThumb,40.72,-73.9845,2533,1,344,148
3,Siempre Verde Garden,"Stanton Street, New York, NY 10002, USA",Community Garden,Outdoor,GreenThumb,40.7199,-73.984,1197,1,344,153
4,Siempre Verde Garden,"Attorney Street, New York, NY 10002, USA",Community Garden,Outdoor,GreenThumb,40.7197,-73.984,1405,1,344,158
...,...,...,...,...,...,...,...,...,...,...,...
2426,35 Saratoga Ave Garden 1,"33-35 Saratoga Ave, Brooklyn, NY 11233, USA",NYCHA Garden,Outdoor,NYCHA,40.686,-73.9176,1110,NaN,NaN,NaN
2427,Butterfly Haven,"344 E 28th St #4c, New York, NY 10016, USA",NYCHA Garden,Outdoor,NYCHA,40.7398,-73.9769,1050,NaN,NaN,NaN
2428,344 East 28th St Garden 2,"342 E 28th St, New York, NY 10016, USA",NYCHA Garden,Outdoor,NYCHA,40.7404,-73.9771,825,NaN,NaN,NaN
2429,303 Vernon Garden 1,"303 Vernon Ave, Brooklyn, NY 11206, USA",NYCHA Garden,Outdoor,NYCHA,40.6962,-73.9401,480,NaN,NaN,NaN


In [ ]:
urban_ag.to_csv('urban_ag.csv', index=False)